# Code Autocompletion with n-grams

We frame the code autocompletion task as follows: given a sequence of $n$ tokens (which you can consider $n$ words of code), predict the $n+1$th token. With this interpretation of the problem in mind, we can use a simple n-gram based maximum likelihood predictor as a rough heuristic for code autocompletion. There are many issues with this approach, but it helps build the intuition of *next token prediction.* Next token prediction is the underlying concept behind Large Language Models, and is one approach that we will be using to solve the code autocompletion problem.

---

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import regex as re
from datasets import load_dataset, Dataset

First, we can start by pulling our dataset.

In [ ]:
def get_data(n: int) -> Dataset:
    """Pull n samples of the codeparrot dataset onto memory, after filtering for Python code."""
    # https://huggingface.co/datasets/codeparrot/github-code
    # Load the dataset
    ds = load_dataset("codeparrot/github-code", streaming=True, split="train")

    # TODO: Filter the dataset for only Python code
    ...

    ds = ds.take(n)

    return ds

# We will only load 1,000 samples for now
dataset = get_data(1_000)

next(iter(dataset))

Now, we need to tokenize our dataset, as the individual "grams" in our n-gram model is built upon tokens. If you are unfamiliar with tokenization, [this](https://medium.com/@abdallahashraf90x/tokenization-in-nlp-all-you-need-to-know-45c00cfa2df7) is a quick read and [this is](https://youtu.be/zduSFxRajkE?si=4TAlVacyZTNUmLn9) a long but incredibly comprehensive watch that you will not regret (Andrej Karpathy is the goat). If you are familiar with tokenization, you likely do not appreciate it enough.

There is a lot to consider when tokenizing a code dataset. For instance, we cannot just naively split based on whitespace - newlines and punctuation are quite important in code, and we may want them to be treated as separate tokens.

In [ ]:
all_tokens = []

# TODO: Tokenize the code snippets
...

In [ ]:
# Quick look at the tokens produced
for token in all_tokens[410:450]:
    print(token)

Now, we can build out our n-grams. In this notebook, we will use 3-grams, and only 3-grams. A better approach would be to leverage a mixture of uni, bi, tri, ..., n grams.

In [ ]:
def get_ngrams(n: int, tokens: list) -> defaultdict:
    """Given a list of tokens, return a dictionary of all the n-grams from the tokens."""
    ngrams = []

    # TODO: Implement the n-gram generation here
    ...

    return ngrams

three_grams = get_ngrams(3, all_tokens)

Based on all of the 3-grams we have derived, we can build a simple count based model we can use to predict the next token given the past 2 tokens.

In [ ]:
model_3gram = ... # TODO: Initialize the 3gram model

for three_gram in three_grams:
    # TODO: Update the model with the 3-grams
    ...

In [ ]:
model_3gram[("def", "main")]

In [ ]:
def plot_model(model: dict, w1: str, w2: str, top_n=10):
    """Plot the top_n words that follow the bigram (w1, w2) in the model."""
    words = [w for w, _ in sorted(model[(w1, w2)].items(), key=lambda x: x[1], reverse=True)[:top_n]]
    counts = [c for _, c in sorted(model[(w1, w2)].items(), key=lambda x: x[1], reverse=True)[:top_n]]
    colors = cm.rainbow(np.linspace(0, 1, len(words)))
    plt.bar(words, counts, color=colors)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
plot_model(model_3gram, "def", "add")

In [ ]:
plot_model(model_3gram, "import", "numpy")

In [ ]:
plot_model(model_3gram, "from", "django")

We can write a simple function that uses the most probable next token to generate a completion based on a provided input.

In [ ]:
def generate(model: dict, w1: str, w2: str, n: int = 10) -> list:
    """Generate the next n token from the model, given the initial bigram (w1, w2)."""
    # TODO: Implement the generation here
    ...

In [ ]:
input_text = "def main"
input_text = input_text.split()
generate(model_3gram, *input_text, n=10)

In [ ]:
input_text = "import pandas"
input_text = input_text.split()
generate(model_3gram, *input_text, n=10)

You should sensible output using the 3-gram model.

Now, on your own, try to use a mixture of different n-gram models to produce better output.

In [ ]:
# TODO: Build other n-gram models, generate text from them,
# and eventually combine them to generate better snippets of code.
...